In [7]:
# disable pycache
import sys
sys.dont_write_bytecode = True

from pudu.apis.foxx_api import *
from pudu.apis.pudu_api import *
from pudu.app.main import App
from pudu.rds import RDSTable
from pudu.rds.utils import *
from pudu.notifications.change_detector import *
from pudu.reporting import *

In [ ]:
get_schedule_table

In [2]:
config_paths = [
        'database_config.yaml',
        '../src/pudu/configs/database_config.yaml',
        'src/pudu/configs/database_config.yaml',
        'pudu/configs/database_config.yaml',
        '/opt/database_config.yaml'
    ]

config_path = None
for path in config_paths:
    if os.path.exists(path):
        config_path = path
        break
if not config_path:
    raise FileNotFoundError("Configuration file not found")


In [5]:
form_data={'customStartDate': '2025-08-20', 'customEndDate': '2025-09-09'}
if form_data.get('customStartDate'):
    print(form_data.get('customStartDate'))
else:
    print('no custom start date')


2025-08-20


In [ ]:

test_form_data = {
    'service': 'robot-management',
    'contentCategories': ['charging-performance', 'cleaning-performance', 'resource-utilization', 'financial-performance'],
    'timeRange': 'custom',
    "location": {
        "country": "us",
        "state": "fl",
        "city": "gainesville"
    },
    'outputFormat': 'html',
    'reportName': 'UF Report',
    'customStartDate': '2025-08-20',
    'customEndDate': '2025-09-09',
    'detailLevel': 'detailed',
    'delivery': 'in-app',
    'schedule': 'immediate'
}
config = ReportConfig(test_form_data, 'test-customer-123')
generator = ReportGenerator(config_path)

result = generator.generate_and_save_report(config, output_format="html", save_file=True)
print(f"Report generation result: {result['success']}")
if result['success']:
    print(f"Report metadata: {result['metadata']}")
else:
    print(f"Error: {result['error']}")

In [ ]:
result['comprehensive_metrics']

{'fleet_performance': {'robots_online_rate': 100.0,
  'total_running_hours': 92.1,
  'total_robots': 2,
  'robots_online': 2,
  'average_robot_utilization': 46.0,
  'avg_task_duration_minutes': 49.3,
  'avg_daily_running_hours_per_robot': 2.7,
  'days_with_tasks': 20,
  'period_length': 21,
  'days_ratio': '20/21'},
 'task_performance': {'total_tasks': 112,
  'completed_tasks': 83,
  'cancelled_tasks': 23,
  'interrupted_tasks': 4,
  'completion_rate': 74.1,
  'total_area_cleaned': 50533.0,
  'coverage_efficiency': 83.4,
  'task_modes': {'Scrubbing': 67, 'Sweeping': 45},
  'incomplete_task_rate': 24.1,
  'weekend_schedule_completion': 55.6,
  'avg_task_duration_minutes': 49.3},
 'charging_performance': {'total_sessions': 121,
  'avg_charging_duration_minutes': 350.7,
  'median_charging_duration_minutes': 141.0,
  'avg_power_gain_percent': 22.5,
  'median_power_gain_percent': 15.0,
  'total_charging_time': 38222},
 'resource_utilization': {'total_energy_consumption_kwh': 34.4,
  'total_

In [ ]:
table = RDSTable(
    connection_config="credentials.yaml",
    database_name="university_of_florida",
    table_name="mnt_robots_task",
    fields=None,
    primary_keys=['robot_sn', 'task_name', 'start_time']
)

In [ ]:
df_orig = table.query_data_as_df("SELECT * FROM mnt_robots_task WHERE is_report=0")
df = df_orig.copy()
# convert start_time to string
df['start_time'] = df['start_time'].astype(str)
data_list = df.drop(columns=['id']).to_dict(orient='records')
data_list

/Users/jiaxuchen/Foxx/Monitor/pudu_robot/src/pudu/rds/utils.py:266: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, connection)


[{'task_id': '1153524218806681600',
  'robot_sn': '811135422060216',
  'task_name': 'Bld_205_Grd_Task ALL',
  'is_report': 0,
  'mode': 'Scrubbing',
  'sub_mode': 'Unknown',
  'type': 'Custom',
  'vacuum_speed': 'Off',
  'vacuum_suction': 'Off',
  'wash_speed': 'High',
  'wash_suction': 'Medium',
  'wash_water': 'High',
  'map_name': '-1#0#Bld_205_Dental_Sci_Grd',
  'map_url': '',
  'new_map_url': None,
  'actual_area': 765.8,
  'plan_area': 789.49,
  'start_time': '2025-08-27 10:01:42',
  'end_time': Timestamp('2025-08-27 15:05:06'),
  'duration': 4121.0,
  'efficiency': 668.98,
  'remaining_time': 95.0,
  'battery_usage': 0.0,
  'consumption': 0.0,
  'water_consumption': 0,
  'progress': 97.0,
  'status': 'In Progress',
  'create_time': Timestamp('2025-08-27 10:03:35'),
  'update_time': Timestamp('2025-08-27 15:03:35')},
 {'task_id': '1156119514279591936',
  'robot_sn': '811135422060228',
  'task_name': 'Marston Main Areas Clean',
  'is_report': 0,
  'mode': 'Sweeping',
  'sub_mode':

In [ ]:
def query_ids_by_unique_keys(cursor, table_name: str, data_list: list, unique_keys: list, pk_column: str):
    """
    Query primary keys for records using their unique key constraints.
    """
    results = []

    # Build batch query to get all primary keys at once
    where_conditions = []

    def escape_value(value):
        if value is None:
            return "NULL"
        elif isinstance(value, str):
            return f"'{value.replace(chr(39), chr(39)+chr(39))}'" # chr(39) == "'"
        else:
            return f"'{value}'"

    for data in data_list:
        record_conditions = []
        for key in unique_keys:
            if key in data:
                value = data[key]
                if value is None:
                    record_conditions.append(f"{key} IS NULL")
                else:
                    record_conditions.append(f"{key} = {escape_value(value)}")

        if record_conditions:
            where_conditions.append("(" + " AND ".join(record_conditions) + ")")

    if where_conditions:
        where_clause = " OR ".join(where_conditions)
        select_columns = [pk_column] + unique_keys
        query = f"SELECT {', '.join(select_columns)} FROM {table_name} WHERE {where_clause}"
        print(query)

        cursor.execute(query)
        db_results = cursor.fetchall()

        # Create mapping from unique key values to primary key
        pk_map = {}
        for row in db_results:
            pk_value = row[0]
            unique_values = row[1:]
            unique_tuple = tuple(unique_values)
            pk_map[unique_tuple] = pk_value
        print(pk_map)

        # Match each original data record to its primary key
        for data in data_list:
            unique_tuple = tuple(data.get(key) for key in unique_keys)
            print(unique_tuple)
            pk_value = pk_map.get(unique_tuple)
            results.append((data, pk_value))

    return results

In [ ]:
def query_ids_by_unique_keys_2(cursor, table_name: str, data_list: list, unique_keys: list, pk_column: str):
    """
    Query primary keys for records using their unique key constraints.
    """
    results = []
    where_conditions = []

    def escape_value(value):
        if value is None:
            return "NULL"
        elif isinstance(value, str):
            return f"'{value.replace(chr(39), chr(39)+chr(39))}'"
        else:
            return f"'{value}'"

    def normalize_for_comparison(value, field_name=None):
        """Normalize values for tuple comparison - convert timestamps to strings for time fields only"""

        # Define time-related field names that need normalization
        TIME_FIELDS = {'start_time', 'end_time', 'create_time', 'update_time', 'task_time', 'upload_time'}

        # Only process time fields
        if field_name and field_name not in TIME_FIELDS:
            return value

        if hasattr(value, 'strftime'):  # datetime/timestamp object
            return value.strftime('%Y-%m-%d %H:%M:%S')
        elif isinstance(value, str) and field_name in TIME_FIELDS:
            # Only try to parse strings for known time fields
            try:
                import pandas as pd
                dt = pd.to_datetime(value)
                return dt.strftime('%Y-%m-%d %H:%M:%S')
            except:
                return value

        return value

    # Build WHERE conditions (same as before)
    for data in data_list:
        record_conditions = []
        for key in unique_keys:
            if key in data:
                value = data[key]
                if value is None:
                    record_conditions.append(f"{key} IS NULL")
                else:
                    record_conditions.append(f"{key} = {escape_value(value)}")

        if record_conditions:
            where_conditions.append("(" + " AND ".join(record_conditions) + ")")

    if where_conditions:
        where_clause = " OR ".join(where_conditions)
        select_columns = [pk_column] + unique_keys
        query = f"SELECT {', '.join(select_columns)} FROM {table_name} WHERE {where_clause}"

        cursor.execute(query)
        db_results = cursor.fetchall()

        # Create mapping with NORMALIZED values
        pk_map = {}
        for row in db_results:
            pk_value = row[0]
            unique_values = row[1:]
            # NORMALIZE database values for comparison
            normalized_tuple = tuple(
                    normalize_for_comparison(val, unique_keys[i])
                    for i, val in enumerate(unique_values)
                )
            pk_map[normalized_tuple] = pk_value

        # Match with NORMALIZED data values
        for data in data_list:
            # NORMALIZE data values for comparison
            normalized_tuple = tuple(
                normalize_for_comparison(data.get(key), key)
                for key in unique_keys
            )
            pk_value = pk_map.get(normalized_tuple)
            results.append((data, pk_value))

    return results

In [ ]:
query_ids_by_unique_keys(table.cursor, table.table_name, data_list, table.primary_keys, 'id')

SELECT id, robot_sn, task_name, start_time FROM mnt_robots_task WHERE (robot_sn = '811135422060216' AND task_name = 'Bld_205_Grd_Task ALL' AND start_time = '2025-08-27 10:01:42') OR (robot_sn = '811135422060228' AND task_name = 'Marston Main Areas Clean' AND start_time = '2025-08-27 13:48:31')
{('811135422060216', 'Bld_205_Grd_Task ALL', datetime.datetime(2025, 8, 27, 10, 1, 42)): 1991, ('811135422060228', 'Marston Main Areas Clean', datetime.datetime(2025, 8, 27, 13, 48, 31)): 1995}
('811135422060216', 'Bld_205_Grd_Task ALL', '2025-08-27 10:01:42')
('811135422060228', 'Marston Main Areas Clean', '2025-08-27 13:48:31')


[({'task_id': '1153524218806681600',
   'robot_sn': '811135422060216',
   'task_name': 'Bld_205_Grd_Task ALL',
   'is_report': 0,
   'mode': 'Scrubbing',
   'sub_mode': 'Unknown',
   'type': 'Custom',
   'vacuum_speed': 'Off',
   'vacuum_suction': 'Off',
   'wash_speed': 'High',
   'wash_suction': 'Medium',
   'wash_water': 'High',
   'map_name': '-1#0#Bld_205_Dental_Sci_Grd',
   'map_url': '',
   'new_map_url': None,
   'actual_area': 765.8,
   'plan_area': 789.49,
   'start_time': '2025-08-27 10:01:42',
   'end_time': Timestamp('2025-08-27 15:05:06'),
   'duration': 4121.0,
   'efficiency': 668.98,
   'remaining_time': 95.0,
   'battery_usage': 0.0,
   'consumption': 0.0,
   'water_consumption': 0,
   'progress': 97.0,
   'status': 'In Progress',
   'create_time': Timestamp('2025-08-27 10:03:35'),
   'update_time': Timestamp('2025-08-27 15:03:35')},
  None),
 ({'task_id': '1156119514279591936',
   'robot_sn': '811135422060228',
   'task_name': 'Marston Main Areas Clean',
   'is_repor

In [ ]:
query_ids_by_unique_keys(table.cursor, table.table_name, data_list, table.primary_keys, 'id')

[({'task_id': '1153524218806681600',
   'robot_sn': '811135422060216',
   'task_name': 'Bld_205_Grd_Task ALL',
   'is_report': 0,
   'mode': 'Scrubbing',
   'sub_mode': 'Unknown',
   'type': 'Custom',
   'vacuum_speed': 'Off',
   'vacuum_suction': 'Off',
   'wash_speed': 'High',
   'wash_suction': 'Medium',
   'wash_water': 'High',
   'map_name': '-1#0#Bld_205_Dental_Sci_Grd',
   'map_url': '',
   'new_map_url': None,
   'actual_area': 765.8,
   'plan_area': 789.49,
   'start_time': '2025-08-27 10:01:42',
   'end_time': Timestamp('2025-08-27 15:05:06'),
   'duration': 4121.0,
   'efficiency': 668.98,
   'remaining_time': 95.0,
   'battery_usage': 0.0,
   'consumption': 0.0,
   'water_consumption': 0,
   'progress': 97.0,
   'status': 'In Progress',
   'create_time': Timestamp('2025-08-27 10:03:35'),
   'update_time': Timestamp('2025-08-27 15:03:35')},
  1991),
 ({'task_id': '1156119514279591936',
   'robot_sn': '811135422060228',
   'task_name': 'Marston Main Areas Clean',
   'is_repor

SELECT id, robot_sn, task_name, start_time FROM mnt_robots_task WHERE (robot_sn = '811135422060216' AND task_name = 'Bld_205_Grd_Task ALL' AND start_time = '2025-08-27 10:01:42') OR (robot_sn = '811135422060228' AND task_name = 'Marston Main Areas Clean' AND start_time = '2025-08-27 10:01:42')


SELECT id, robot_sn, task_name, start_time FROM mnt_robots_task WHERE (robot_sn = '811135422060216' AND task_name = 'Bld_205_Grd_Task ALL' AND start_time = '2025-08-27 10:01:42') OR (robot_sn = '811135422060228' AND task_name = 'Marston Main Areas Clean' AND start_time = '2025-08-27 10:01:42')


In [ ]:
'''
From pudu, the robot position is in the robot map coordinate system.
Transform the robot position to the pixel on the robot map and the floor plan.

Yongming
2025-08-17
'''
from sqlalchemy.pool import NullPool
import yaml
import os
from sqlalchemy import create_engine, text
from datetime import datetime
import numpy as np
import time
import requests
from PIL import Image
import io
import cv2
import json
import xml.etree.ElementTree as ET

'''

status:
database columns: id, robot_sn, map_name, x, y, z, status, update_time

task:
database columns: id, task_id, robot_sn, task_name, mode, sub_mode, `type`, vacuum_speed, vacuum_suction, wash_speed, wash_suction, wash_water, map_name, map_url, actual_area, plan_area, start_time, end_time, duration, efficiency, remaining_time, consumption, water_consumption, progress, status, create_time, update_time

robot_sn:
811135422060228 --> marston-science-library
811135422060216 --> dental-science-building

'''
class TransformToFloorPlan:
    def __init__(self):

        config_file = "pudu/rds/credentials.yaml"
        self.database_name = "university_of_florida"

        with open(config_file, 'r') as file:
            config = yaml.safe_load(file)
            db_config = config['database']

        from pudu.rds.utils import get_secret
        username, password = get_secret(db_config['secret_name'], db_config['region_name'])

        # Use the clean database name (without backticks) for connection string
        clean_db_name = self.database_name.strip('`')

        # Create connection string with database
        connection_string = f"mysql+pymysql://{username}:{password}@{db_config['host']}/{clean_db_name}"

        self.engine = create_engine(
            connection_string,
            poolclass=NullPool,
            echo=False,
            future=True  # Use SQLAlchemy 2.0 style
        )
    #################################################
    ############### robot task report ###############
    #################################################

    def convert_task_report_to_floor_plan(self, map_name: str, map_url: str):
        # 1) Fetch images
        if not map_url or not map_name:
            print(f'map_url or map_name is {map_url} or {map_name}')
            return None
        print(f'map_url: {map_url}, map_name: {map_name}')
        task_report_png_bytes = self.fetch_png_from_s3_url(map_url)
        map_info = self.fetch_map_info(map_name)
        floor_plan_png_bytes = self.fetch_png_from_s3_url(map_info['floor_map'])

        # Read as RGB via PIL (consistent with your green check)
        task_img_rgb = np.array(Image.open(io.BytesIO(task_report_png_bytes)).convert('RGB'))
        floor_img_rgb = np.array(Image.open(io.BytesIO(floor_plan_png_bytes)).convert('RGB'))

        # 2) Green mask in task image (exact color match)
        green = np.array([28, 195, 61], dtype=np.uint8)
        green_mask = np.all(task_img_rgb == green, axis=-1).astype(np.uint8)  # HxW, {0,1}

        # 3) Load transform (3x3 homography: task -> floor)
        transform_task_to_floor = map_info['transform_task_to_floor']

        # 4) Warp the green mask into floor-plan coordinates
        h_floor, w_floor = floor_img_rgb.shape[:2]
        # OpenCV expects size=(width, height)
        warped_mask = cv2.warpPerspective(
            green_mask, transform_task_to_floor, (w_floor, h_floor),
            flags=cv2.INTER_NEAREST  # keep mask crisp
        )  # HxW, float32/uint8

        # 5) Optional: dilate/blur mask to make overlay more visible (tweak as needed)
        # kernel = np.ones((3, 3), np.uint8)
        # warped_mask = cv2.dilate(warped_mask, kernel, iterations=1)

        # 6) Create a colored overlay (green) where the mask is 1
        overlay = floor_img_rgb.copy()
        overlay[warped_mask == 1] = green  # paint pure green on those pixels

        # 7) Alpha blend overlay with the original floor plan
        alpha = 0.5  # 0..1
        blended = (overlay.astype(np.float32) * alpha +
                  floor_img_rgb.astype(np.float32) * (1 - alpha)).astype(np.uint8)

        # 8) Save resulting PNG (convert to bytes if your save expects bytes)
        # If self.save_png expects a NumPy array, this is fine:
        self.save_png(blended, 'green_on_floor_plan.png') #yq for debug

        return blended

    def save_png(self, image: np.ndarray, file_name: str):
        img = Image.fromarray(image)
        img.save(file_name)
        print(f'Saved {file_name}')

    # Convert the robot position from floor plan to occupancy grid map
    def fetch_map_info(self, robot_map_name: str):
        with self.engine.connect() as conn:
            row = conn.execute(text('''
                SELECT transform_robot_to_floor, transform_task_to_floor, floor_map, robot_map, robot_map_xml
                FROM pro_floor_info
                WHERE robot_map_name = :robot_map_name
            '''), {'robot_map_name': robot_map_name}).fetchone()
            new_row = {}
            new_row['transform_robot_to_floor'] = np.array(json.loads(row.transform_robot_to_floor)).reshape(3, 3) if row.transform_robot_to_floor is not None else None
            new_row['transform_task_to_floor'] = np.array(json.loads(row.transform_task_to_floor)).reshape(3, 3) if row.transform_task_to_floor is not None else None
            new_row['floor_map'] = row.floor_map
            new_row['robot_map'] = row.robot_map
            new_row['robot_map_xml'] = row.robot_map_xml
            return new_row

    # Query the robot task from the database
    def test_robot_task(self, robot_sn: str):
        with self.engine.connect() as conn:
            row = conn.execute(text('''
                SELECT map_name, map_url, id
                FROM mnt_robots_task
                WHERE robot_sn = :robot_sn
                ORDER BY start_time DESC
                LIMIT 1 OFFSET 1
            '''), {'robot_sn': robot_sn}).fetchone()
            if row is None:
                return None

            task_png_url = row.map_url
            map_name = row.map_name
            print(f'task_png_url: {task_png_url}, map_name: {map_name}, id: {row.id} in function test_robot_task')
            if task_png_url and map_name:
                self.convert_task_report_to_floor_plan(map_name, task_png_url)
            return row

    def fetch_png_from_s3_url(self, s3_url: str):
        response = requests.get(s3_url)
        if response.status_code == 200:
            return response.content
        else:
            print(f'Failed to fetch PNG from {s3_url}, status code: {response.status_code}')
            return None

    #################################################
    ############### robot position ##################
    #################################################
    # Convert the robot position of x, y to the pixel on the robot map (x, y) and on the floor plan (new_x, new_y)
    def position_to_pixel(self, map_name: str, x: float, y: float):
        map_info = self.fetch_map_info(map_name)
        # Load the robot map and the floor plan
        robot_map_png_bytes = self.fetch_png_from_s3_url(map_info['robot_map'])
        floor_plan_png_bytes = self.fetch_png_from_s3_url(map_info['floor_map'])
        robot_map_rgb = np.array(Image.open(io.BytesIO(robot_map_png_bytes)).convert('RGB'))
        floor_plan_rgb = np.array(Image.open(io.BytesIO(floor_plan_png_bytes)).convert('RGB'))

        # Get the resolution and origin of the robot map
        # Transform the robot position to the robot map
        robot_map_xml = map_info['robot_map_xml']
        robot_map_xml = ET.fromstring(robot_map_xml)
        resolution = float(robot_map_xml.find('resolution').text)
        origin = list(map(float, robot_map_xml.find('origin').text.split()))
        robot_map_u = int((x - origin[0]) / resolution)
        robot_map_v = int(robot_map_rgb.shape[0] - (y - origin[1]) / resolution)

        # Transform the robot position to the floor plan
        transform_robot_to_floor = np.array(map_info['transform_robot_to_floor'], dtype=np.float64).reshape(3, 3)
        robot_position_on_floor = transform_robot_to_floor @ np.array([robot_map_u, robot_map_v, 1])
        floor_plan_u = int(robot_position_on_floor[0])
        floor_plan_v = int(robot_position_on_floor[1])
        return robot_map_u, robot_map_v,floor_plan_u, floor_plan_v, robot_map_rgb, floor_plan_rgb


    # Query the robot status from the database
    def test_robot_status(self, map_name: str, x: float, y: float):
        robot_map_u, robot_map_v, floor_plan_u, floor_plan_v, robot_map_rgb, floor_plan_rgb = self.position_to_pixel(map_name, x, y)
        # robot_map_u = 612
        # robot_map_v = 263
        # floor_plan_u = 530.0
        # floor_plan_v = 1281.0
        marked_robot_map_img = cv2.circle(robot_map_rgb, (robot_map_u, robot_map_v), 10, (0, 0, 255), -1)
        marked_floor_plan_img = cv2.circle(floor_plan_rgb, (floor_plan_u, floor_plan_v), 10, (0, 0, 255), -1)
        self.save_png(marked_robot_map_img, 'robot_position_on_robot_map.png')
        self.save_png(marked_floor_plan_img, 'robot_position_on_floor_plan.png')
        print(robot_map_u, robot_map_v, floor_plan_u, floor_plan_v)
        return robot_map_u, robot_map_v, floor_plan_u, floor_plan_v


In [ ]:
transformer_orig = TransformToFloorPlan()

In [ ]:
transformer_orig.test_robot_status('-1#0#Bld_205_Dental_Sci_Grd', -68.4046624976004, 61.043604755139825)

Saved robot_position_on_robot_map.png
Saved robot_position_on_floor_plan.png
475 716 2067 1414


(475, 716, 2067, 1414)

In [ ]:
map_name = '2#0#43_Marston_library_2nd'
x =-0.34194893508938584
y =-9.985255865113176

transformer_orig.position_to_pixel(map_name, x, y)

(555, 430, 786, 1383)

-1#0#Bld_205_Dental_Sci_Grd


1#1#dental_ground_elevator, 1#0#Bld_205_Dental_Sci_1fl

-1#1#Bld205_Grd_Elevator, -1#0#Bld205_Grd_Elevator, 1#0#Bld_205_Dental_Sci_1fl, 2#0#Bld205-Floor2

In [ ]:
def find_longest_increasing_subsequence(arr):
    '''dp to find the length of the longest increasing subsequence as well as list of indices that make up the subsequence'''
    LIS = [1] * len(arr)
    prev = [-1] * len(arr)

    for i in range(len(arr)):
        for j in range(i):
            if arr[i] > arr[j] and LIS[j] + 1 > LIS[i]:
                LIS[i] = LIS[j] + 1
                prev[i] = j

    max_len = max(LIS)
    # find the index of the last element in the longest increasing subsequence
    last_index = LIS.index(max_len)
    # backtrack to find the subsequence
    subsequence = []
    while last_index != -1:
        subsequence.append(last_index)
        last_index = prev[last_index]
    return max_len, subsequence[::-1]

max_len, subsequence = find_longest_increasing_subsequence([2, -1, 3, 2, 0, 5, -3, 4, 7])
max_len, subsequence


(4, [0, 2, 5, 8])

In [8]:
df = get_schedule_table(start_time='2025-08-21 00:00:00', end_time='2025-08-22 23:59:59')
df.head()

,Location ID,Task Name,Task ID,Robot SN,Map Name,Is Report,Map URL,Actual Area,Plan Area,Start Time,...,Progress,Status,Mode,Sub Mode,Type,Vacuum Speed,Vacuum Suction,Wash Speed,Wash Suction,Wash Water
0,533370001,Bld_205_1st_Task ALL,1153255765814296576,811135422060216,1#0#Bld_205_Dental_Sci_1fl,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,783.78,783.78,2025-08-22 21:08:31,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,Standard,Low,High
1,533370001,Marston Lib 1st vac+sweep,1153144660123205632,811135422060228,1#0#43_Marston_Library_F1,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,948.30,948.30,2025-08-22 12:52:23,...,100,Task Ended,Sweeping,Custom,Custom,Standard,Medium,Off,Off,Off
2,533370001,ground_elevator_new,1153905339323138048,811135422060216,1#1#dental_ground_elevator,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,102.32,102.32,2025-08-22 14:46:01,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,High
3,533370001,Bld_205_Grd_Task ALL,1153524218806681600,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,789.49,789.49,2025-08-22 11:50:58,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,High,Medium,High
4,533370001,Marston Lib 1st vac+sweep,1153144660123205632,811135422060228,1#0#43_Marston_Library_F1,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,0.00,948.30,2025-08-22 11:27:42,...,0,Task Cancelled,Sweeping,Custom,Custom,Standard,Medium,Off,Off,Off


In [11]:
map_name = "Bld_205_Dental_Sci_Grd"

# split on '#' but only the first two
parts = map_name.split("#", 2)

# take the remainder after the first two '#'
cleaned = parts[2] if len(parts) > 2 else map_name

print(parts)  # Bld_205_Dental_Sci_Grd

['Bld_205_Dental_Sci_Grd']


In [ ]:
df.shape

(10, 27)

In [ ]:
df.loc[df['Robot SN'] == '811135422060228', 'Progress']

,Location ID,Task Name,Task ID,Robot SN,Map Name,Is Report,Map URL,Actual Area,Plan Area,Start Time,...,Progress,Status,Mode,Sub Mode,Type,Vacuum Speed,Vacuum Suction,Wash Speed,Wash Suction,Wash Water
0,533370001,Marston 1st main area,1156100454242336768,811135422060228,1#0#43_Marston_Library_F1,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,164.01,431.6,2025-08-21 18:51:57,...,38,Task Cancelled,Sweeping,Unknown,Unknown,Standard,Medium,Off,Off,Off
2,533370001,Marston Lib 1st vac+sweep,1153144660123205632,811135422060228,1#0#43_Marston_Library_F1,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,948.30,948.3,2025-08-21 13:23:00,...,100,Task Ended,Sweeping,Custom,Custom,Standard,Medium,Off,Off,Off
9,533370001,Marston lib 2nd sweep+vac,1153563046208618496,811135422060228,2#0#43_Marston_library_2nd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,697.80,697.8,2025-08-21 09:27:34,...,100,Task Ended,Sweeping,Unknown,Unknown,Standard,Medium,Off,Off,Off


In [ ]:
get_robot_work_location_and_mapping_data()

(          robot_sn    map_name         x         y        z  status  \
 0  8110H4802050006        None       NaN       NaN      NaN    idle   
 1  8110H4802050005  1#4#church  0.633748  0.113463 -0.20346  normal   
 2  8110H4B08050040        None       NaN       NaN      NaN    idle   
 3  811135422060217        None       NaN       NaN      NaN    idle   
 4  811135422060228        None       NaN       NaN      NaN    idle   
 5  811135422060216        None       NaN       NaN      NaN    idle   
 
            update_time  
 0  2025-08-21 17:37:26  
 1  2025-08-21 17:37:26  
 2  2025-08-21 17:37:26  
 3  2025-08-21 17:37:26  
 4  2025-08-21 17:37:26  
 5  2025-08-21 17:37:26  ,
      map_name floor_number
 0  1#4#church            1)

In [ ]:
import os
config_paths = [
        'database_config.yaml',
        '../src/pudu/configs/database_config.yaml',
        'src/pudu/configs/database_config.yaml',
        'pudu/configs/database_config.yaml',
        '/opt/database_config.yaml'
    ]

config_path = None
for path in config_paths:
    if os.path.exists(path):
        config_path = path
        break
if not config_path:
    raise FileNotFoundError("Configuration file not found")

# Initialize app
app = App(config_path=config_path)

In [ ]:
processed_data = app._prepare_df_for_database(df, columns_to_remove=['location_id'])
processed_data

,task_name,task_id,robot_sn,map_name,is_report,map_url,actual_area,plan_area,start_time,end_time,...,progress,status,mode,sub_mode,type,vacuum_speed,vacuum_suction,wash_speed,wash_suction,wash_water
0,Marston 1st main area,1156100454242336768,811135422060228,1#0#43_Marston_Library_F1,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,164.01,431.60,2025-08-21 18:51:57,2025-08-21 19:14:49,...,38,Task Cancelled,Sweeping,Unknown,Unknown,Standard,Medium,Off,Off,Off
1,ground_elevator_new,1153905339323138048,811135422060216,1#1#dental_ground_elevator,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,102.32,102.32,2025-08-21 17:00:51,2025-08-21 17:22:29,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,High
2,Marston Lib 1st vac+sweep,1153144660123205632,811135422060228,1#0#43_Marston_Library_F1,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,948.30,948.30,2025-08-21 13:23:00,2025-08-21 15:37:40,...,100,Task Ended,Sweeping,Custom,Custom,Standard,Medium,Off,Off,Off
3,Bld_205_Grd_Task dock heavy,1153158890205495296,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,105.16,105.16,2025-08-21 12:53:58,2025-08-21 13:09:51,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,High,High,High
4,Bld_205_Grd_Task dock heavy,1153158890205495296,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,0.00,105.16,2025-08-21 12:52:05,2025-08-21 12:52:38,...,0,Task Cancelled,Scrubbing,Custom,Custom,Off,Off,High,High,High
5,Bld_205_Grd_Task dock heavy,1153158890205495296,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,0.00,105.16,2025-08-21 12:48:10,2025-08-21 12:51:28,...,0,Task Cancelled,Scrubbing,Custom,Custom,Off,Off,High,High,High
6,ground_elevator_new,1153905339323138048,811135422060216,1#1#dental_ground_elevator,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,102.32,102.32,2025-08-21 12:19:02,2025-08-21 12:39:30,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,High
7,ground_elevator_new,1153905339323138048,811135422060216,1#1#dental_ground_elevator,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,1.02,102.32,2025-08-21 12:16:52,2025-08-21 12:18:02,...,1,Task Cancelled,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,Medium
8,Bld_205_Grd_Task ALL,1153524218806681600,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,789.49,789.49,2025-08-21 09:32:01,2025-08-21 11:17:06,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,High,Medium,High
9,Marston lib 2nd sweep+vac,1153563046208618496,811135422060228,2#0#43_Marston_library_2nd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,697.80,697.80,2025-08-21 09:27:34,2025-08-21 11:00:36,...,100,Task Ended,Sweeping,Unknown,Unknown,Standard,Medium,Off,Off,Off


In [ ]:
    # Check if config file exists
    config_paths = [
        'database_config.yaml',
        '../src/pudu/configs/database_config.yaml',
        'src/pudu/configs/database_config.yaml',
        'pudu/configs/database_config.yaml',
        '../configs/database_config.yaml'
    ]

    config_path = None
    for path in config_paths:
        if os.path.exists(path):
            config_path = path
            break
    if not config_path:
        raise FileNotFoundError("Configuration file not found")

In [ ]:
app._filter_tasks_needing_transformation(processed_data)

,task_name,task_id,robot_sn,map_name,is_report,map_url,actual_area,plan_area,start_time,end_time,...,progress,status,mode,sub_mode,type,vacuum_speed,vacuum_suction,wash_speed,wash_suction,wash_water
3,Bld_205_Grd_Task dock heavy,1153158890205495296,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,105.16,105.16,2025-08-21 12:53:58,2025-08-21 13:09:51,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,High,High,High
4,Bld_205_Grd_Task dock heavy,1153158890205495296,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,0.00,105.16,2025-08-21 12:52:05,2025-08-21 12:52:38,...,0,Task Cancelled,Scrubbing,Custom,Custom,Off,Off,High,High,High
5,Bld_205_Grd_Task dock heavy,1153158890205495296,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,0.00,105.16,2025-08-21 12:48:10,2025-08-21 12:51:28,...,0,Task Cancelled,Scrubbing,Custom,Custom,Off,Off,High,High,High
8,Bld_205_Grd_Task ALL,1153524218806681600,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,789.49,789.49,2025-08-21 09:32:01,2025-08-21 11:17:06,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,High,Medium,High


In [ ]:
df = get_schedule_table(start_time='2025-08-20 00:00:00', end_time='2025-08-20 23:59:59')
df.head()

,Location ID,Task Name,Task ID,Robot SN,Map Name,Is Report,Map URL,Actual Area,Plan Area,Start Time,...,Progress,Status,Mode,Sub Mode,Type,Vacuum Speed,Vacuum Suction,Wash Speed,Wash Suction,Wash Water
0,533370001,Bld_205_1st_Task ALL,1153255765814296576,811135422060216,1#0#Bld_205_Dental_Sci_1fl,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,783.78,783.78,2025-08-20 21:40:15,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,Standard,Low,High
1,533370001,Marston Lib 1st vac+sweep,1153144660123205632,811135422060228,1#0#43_Marston_Library_F1,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,948.30,948.30,2025-08-20 13:34:20,...,100,Task Ended,Sweeping,Custom,Custom,Standard,Medium,Off,Off,Off
2,533370001,Bld_205_Grd_Task ALL,1153524218806681600,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,513.17,789.49,2025-08-20 13:09:37,...,65,Task Interrupted,Scrubbing,Custom,Custom,Off,Off,High,Medium,High
3,533370001,ground_elevator_new,1153905339323138048,811135422060216,1#1#dental_ground_elevator,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,102.32,102.32,2025-08-20 12:15:19,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,Medium
4,533370001,Bld_205_Grd_Task ALL,1153524218806681600,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,789.49,789.49,2025-08-20 09:33:28,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,High,Medium,High


In [ ]:
processed_data = app._prepare_df_for_database(df, columns_to_remove=['location_id'])
processed_data

,task_name,task_id,robot_sn,map_name,is_report,map_url,actual_area,plan_area,start_time,end_time,...,progress,status,mode,sub_mode,type,vacuum_speed,vacuum_suction,wash_speed,wash_suction,wash_water
0,Bld_205_1st_Task ALL,1153255765814296576,811135422060216,1#0#Bld_205_Dental_Sci_1fl,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,783.78,783.78,2025-08-20 21:40:15,2025-08-20 23:13:30,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,Standard,Low,High
1,Marston Lib 1st vac+sweep,1153144660123205632,811135422060228,1#0#43_Marston_Library_F1,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,948.30,948.30,2025-08-20 13:34:20,2025-08-20 15:48:05,...,100,Task Ended,Sweeping,Custom,Custom,Standard,Medium,Off,Off,Off
2,Bld_205_Grd_Task ALL,1153524218806681600,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,513.17,789.49,2025-08-20 13:09:37,2025-08-20 14:21:01,...,65,Task Interrupted,Scrubbing,Custom,Custom,Off,Off,High,Medium,High
3,ground_elevator_new,1153905339323138048,811135422060216,1#1#dental_ground_elevator,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,102.32,102.32,2025-08-20 12:15:19,2025-08-20 12:36:19,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,Medium
4,Bld_205_Grd_Task ALL,1153524218806681600,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,789.49,789.49,2025-08-20 09:33:28,2025-08-20 11:18:46,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,High,Medium,High
5,Marston lib 2nd sweep+vac,1153563046208618496,811135422060228,2#0#43_Marston_library_2nd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,697.80,697.80,2025-08-20 09:18:40,2025-08-20 10:52:21,...,100,Task Ended,Sweeping,Unknown,Unknown,Standard,Medium,Off,Off,Off


In [ ]:
transformed_data = app.transform_service.transform_task_maps_batch(processed_data)

⚠️ Error transforming task map for task Bld_205_Grd_Task ALL of robot 811135422060216: 'NoneType' object has no attribute 'astype'
⚠️ Error transforming task map for task Bld_205_Grd_Task ALL of robot 811135422060216: 'NoneType' object has no attribute 'astype'


In [ ]:
transformed_data

,task_name,task_id,robot_sn,map_name,is_report,map_url,actual_area,plan_area,start_time,end_time,...,status,mode,sub_mode,type,vacuum_speed,vacuum_suction,wash_speed,wash_suction,wash_water,new_map_url
0,Bld_205_1st_Task ALL,1153255765814296576,811135422060216,1#0#Bld_205_Dental_Sci_1fl,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,783.78,783.78,2025-08-20 21:40:15,2025-08-20 23:13:30,...,Task Ended,Scrubbing,Custom,Custom,Off,Off,Standard,Low,High,
1,Marston Lib 1st vac+sweep,1153144660123205632,811135422060228,1#0#43_Marston_Library_F1,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,948.30,948.30,2025-08-20 13:34:20,2025-08-20 15:48:05,...,Task Ended,Sweeping,Custom,Custom,Standard,Medium,Off,Off,Off,
2,Bld_205_Grd_Task ALL,1153524218806681600,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,513.17,789.49,2025-08-20 13:09:37,2025-08-20 14:21:01,...,Task Interrupted,Scrubbing,Custom,Custom,Off,Off,High,Medium,High,
3,ground_elevator_new,1153905339323138048,811135422060216,1#1#dental_ground_elevator,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,102.32,102.32,2025-08-20 12:15:19,2025-08-20 12:36:19,...,Task Ended,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,Medium,
4,Bld_205_Grd_Task ALL,1153524218806681600,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,789.49,789.49,2025-08-20 09:33:28,2025-08-20 11:18:46,...,Task Ended,Scrubbing,Custom,Custom,Off,Off,High,Medium,High,
5,Marston lib 2nd sweep+vac,1153563046208618496,811135422060228,2#0#43_Marston_library_2nd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,697.80,697.80,2025-08-20 09:18:40,2025-08-20 10:52:21,...,Task Ended,Sweeping,Unknown,Unknown,Standard,Medium,Off,Off,Off,


In [ ]:
transformed_data.iloc[0]['new_map_url']

'https://pudu-robot-transforms-university-of-florida-796835-us-east-2.s3.us-east-2.amazonaws.com/transformed-maps/1_0_Bld_205_Dental_Sci_1fl/5251c3a1f780.png'

https://pudu-robot-transforms-university-of-florida-796835-us-east-2.s3.us-east-2.amazonaws.com/transformed-maps/1_0_Bld_205_Dental_Sci_1fl/5251c3a1f780.png
https://pudu-robot-transforms-university-of-florida-796835-us-east-2.s3.us-east-2.amazonaws.com/transformed-maps/1_0_Bld_205_Dental_Sci_1fl/5251c3a1f780.png

In [ ]:
# from pudu.services.work_location_service import WorkLocationService
# work_location_service = WorkLocationService()
work_location_data, mapping_data = get_robot_work_location_and_mapping_data()

In [ ]:
work_location_data

,robot_sn,map_name,x,y,z,status,update_time
0,8110H4802050006,None,NaN,NaN,NaN,idle,2025-08-21 12:03:37
1,8110H4802050005,1#4#church,0.633748,0.113463,-0.203460,normal,2025-08-21 12:03:37
2,8110H4B08050040,None,NaN,NaN,NaN,idle,2025-08-21 12:03:37
3,811135422060217,None,NaN,NaN,NaN,idle,2025-08-21 12:03:37
4,811135422060228,1#0#43_Marston_Library_F1,-26.135264,10.961617,-0.745841,normal,2025-08-21 12:03:37
5,811135422060216,None,NaN,NaN,NaN,idle,2025-08-21 12:03:37


In [ ]:
transformed_data = app.transform_service.transform_robot_coordinates_batch(work_location_data)

In [ ]:
transformed_data

,robot_sn,map_name,x,y,z,status,update_time,new_x,new_y
0,8110H4802050006,None,NaN,NaN,NaN,idle,2025-08-21 12:03:37,None,None
1,8110H4802050005,1#4#church,0.633748,0.113463,-0.203460,normal,2025-08-21 12:03:37,None,None
2,8110H4B08050040,None,NaN,NaN,NaN,idle,2025-08-21 12:03:37,None,None
3,811135422060217,None,NaN,NaN,NaN,idle,2025-08-21 12:03:37,None,None
4,811135422060228,1#0#43_Marston_Library_F1,-26.135264,10.961617,-0.745841,normal,2025-08-21 12:03:37,253.0,640.0
5,811135422060216,None,NaN,NaN,NaN,idle,2025-08-21 12:03:37,None,None


In [ ]:
transformed_data

,robot_sn,map_name,x,y,z,status,update_time,new_x,new_y
0,8110H4802050006,None,NaN,NaN,NaN,idle,2025-08-21 10:41:19,None,None
1,8110H4802050005,1#4#church,0.633748,0.113463,-0.203460,normal,2025-08-21 10:41:19,None,None
2,8110H4B08050040,None,NaN,NaN,NaN,idle,2025-08-21 10:41:19,None,None
3,811135422060217,None,NaN,NaN,NaN,idle,2025-08-21 10:41:19,None,None
4,811135422060228,1#0#43_Marston_Library_F1,3.244316,-15.777315,-2.744504,normal,2025-08-21 10:41:19,1188.0,1514.0
5,811135422060216,None,NaN,NaN,NaN,idle,2025-08-21 10:41:19,None,None


In [ ]:
get_list_maps(shop_id=533370001)

{'count': 7,
 'list': [{'map_name': '1#1#dental_ground_elevator'},
  {'map_name': '2#0#Bld205-Floor2'},
  {'map_name': '-1#1#Bld205_Grd_Elevator'},
  {'map_name': '1#0#Bld_205_Dental_Sci_1fl'},
  {'map_name': '2#0#43_Marston_library_2nd'},
  {'map_name': '-1#0#Bld_205_Dental_Sci_Grd'},
  {'map_name': '1#0#43_Marston_Library_F1'}]}

In [ ]:
df = get_schedule_table(start_time='2025-08-20 00:00:00', end_time='2025-08-20 23:59:59')
df

,Location ID,Task Name,Task ID,Robot SN,Map Name,Is Report,Map URL,Actual Area,Plan Area,Start Time,...,Progress,Status,Mode,Sub Mode,Type,Vacuum Speed,Vacuum Suction,Wash Speed,Wash Suction,Wash Water


In [ ]:
get_ongoing_tasks_table()

,location_id,task_name,task_id,robot_sn,map_name,is_report,map_url,actual_area,plan_area,start_time,...,progress,status,mode,sub_mode,type,vacuum_speed,vacuum_suction,wash_speed,wash_suction,wash_water
0,450270000,gym,1119987424014385152,8110H4802050005,1#4#church,0,,2.49,249.30,2025-08-20 08:57:03,...,1,Task Interrupted,Sweeping,Unknown,Custom,Standard,Medium,Off,Off,Off
1,533370001,Marston Lib 1st vac+sweep,1153144660123205632,811135422060228,1#0#43_Marston_Library_F1,0,,331.91,948.30,2025-08-20 08:19:49,...,35,In Progress,Sweeping,Unknown,Custom,Standard,Medium,Off,Off,Off
2,533370001,Bld_205_1st_Task ALL,1153255765814296576,811135422060216,1#0#Bld_205_Dental_Sci_1fl,0,,752.43,783.78,2025-08-20 07:31:19,...,96,Task Suspended,Scrubbing,Unknown,Custom,Off,Off,Standard,Medium,Medium


In [ ]:
df = get_schedule_table(start_time='2025-08-15 00:00:00', end_time='2025-08-15 23:59:59')
df.columns = [col.lower().replace(' ', '_') for col in df.columns]
df

,location_id,task_name,task_id,robot_sn,map_name,is_report,map_url,actual_area,plan_area,start_time,...,status,mode,sub_mode,type,vacuum_speed,vacuum_suction,wash_speed,wash_suction,wash_water,battery_usage
0,533370001,Bld_205_1st_Task ALL,1153255765814296576,811135422060216,1#0#Bld_205_Dental_Sci_1fl,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,783.78,783.78,2025-08-15 22:14:55,...,Task Ended,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,Medium,33.0
1,533370001,ground_elevator_new,1153905339323138048,811135422060216,1#1#dental_ground_elevator,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,102.32,102.32,2025-08-15 18:06:43,...,Task Ended,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,Medium,9.0
2,533370001,Bld_205_Grd_Task 1-3,1153152591820505088,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,52.14,260.71,2025-08-15 17:46:17,...,Task Cancelled,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,Medium,4.0
3,533370001,ground_elevator_new,1153905339323138048,811135422060216,1#1#dental_ground_elevator,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,6.14,102.32,2025-08-15 17:30:24,...,Task Cancelled,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,Medium,0.0
4,533370001,Marston Lib 1st vac+sweep,1153144660123205632,811135422060228,1#0#43_Marston_Library_F1,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,739.67,948.30,2025-08-15 11:51:05,...,Task Ended,Sweeping,Custom,Custom,Standard,Medium,Off,Off,Off,61.0
5,533370001,Bld_205_Grd_Task ALL,1153524218806681600,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,793.52,793.52,2025-08-15 12:12:03,...,Task Ended,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,Medium,41.0
6,533370001,Marston Lib 1st vac+sweep,1153144660123205632,811135422060228,1#0#43_Marston_Library_F1,1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,948.30,948.30,2025-08-15 09:11:06,...,Task Ended,Sweeping,Custom,Custom,Standard,Medium,Off,Off,Off,55.0


In [ ]:
table = RDSTable(
    connection_config="credentials.yaml",
    database_name="university_of_florida",
    table_name="mnt_robots_task",
    fields=None,
    primary_keys=['robot_sn', 'task_name', 'start_time']
)

In [ ]:
df = table.query_data_as_df("SELECT * FROM mnt_robots_task")
df.head(1)

/Users/jiaxuchen/Foxx/Monitor/pudu_robot/src/pudu/rds/utils.py:264: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, connection)


,id,task_id,robot_sn,task_name,is_report,mode,sub_mode,type,vacuum_speed,vacuum_suction,...,duration,efficiency,remaining_time,battery_usage,consumption,water_consumption,progress,status,create_time,update_time
0,1566,1153611826161270784,811135422060216,Bld205 Floor2 All,1,Scrubbing,Custom,Custom,Off,Off,...,0.0,0.0,0.0,0.0,0.0,0,0.0,Task Cancelled,2025-08-15 13:23:35,2025-08-20 05:27:47


In [ ]:
data_list = df.drop(columns=['id']).to_dict(orient='records')
data_list_new = data_list.copy()[0]
data_list_new['task_namess'] = '123'
data_list_new['task_name'] = 'mode1'
changes = detect_data_changes(table, [data_list_new], table.primary_keys)
changes

{'811135422060216_mode1_2025-08-14 22:03:08': {'robot_sn': '811135422060216',
  'primary_key_values': {'robot_sn': '811135422060216',
   'task_name': 'mode1',
   'start_time': Timestamp('2025-08-14 22:03:08')},
  'change_type': 'new_record',
  'changed_fields': ['task_id',
   'robot_sn',
   'task_name',
   'is_report',
   'mode',
   'sub_mode',
   'type',
   'vacuum_speed',
   'vacuum_suction',
   'wash_speed',
   'wash_suction',
   'wash_water',
   'map_name',
   'map_url',
   'new_map_url',
   'actual_area',
   'plan_area',
   'start_time',
   'end_time',
   'duration',
   'efficiency',
   'remaining_time',
   'battery_usage',
   'consumption',
   'water_consumption',
   'progress',
   'status',
   'create_time',
   'update_time',
   'task_namess'],
  'old_values': {},
  'new_values': {'task_id': '1153611826161270784',
   'robot_sn': '811135422060216',
   'task_name': 'mode1',
   'is_report': 1,
   'mode': 'Scrubbing',
   'sub_mode': 'Custom',
   'type': 'Custom',
   'vacuum_speed': 

In [ ]:
get_robot_details('811135422060216')

{'battery': 39,
 'cleanbot': {'clean': None,
  'detail': '',
  'last_mode': 1,
  'last_task': '1153905339323138048',
  'rising': 25,
  'sewage': 25,
  'task': 0},
 'mac': 'AC:D9:29:88:F8:37',
 'map': {'floor': '', 'lv': 11, 'name': '1#1#dental_ground_elevator'},
 'nickname': 'Building_205_Dental_Science',
 'online': True,
 'position': {'x': 0.1292594298033276,
  'y': 0.030330103413005904,
  'z': 3.014894040775217},
 'shop': {'id': 533370001, 'name': 'University of Florida'},
 'sn': '811135422060216'}

In [ ]:
get_robot_details('811135422060228')

{'battery': 100,
 'cleanbot': {'clean': None,
  'detail': '',
  'last_mode': 2,
  'last_task': '1156119514279591936',
  'rising': 0,
  'sewage': 0,
  'task': 202},
 'mac': 'AC:D9:29:88:F8:BF',
 'map': {'floor': '', 'lv': 7, 'name': '1#1#43_Main_areaMarston_Library_F1'},
 'nickname': 'Building_43_Marston_Library',
 'online': True,
 'position': {'x': -0.15776518005748486,
  'y': 0.016152815434324452,
  'z': -0.0031088557271414796},
 'shop': {'id': 533370001, 'name': 'University of Florida'},
 'sn': '811135422060228'}

In [ ]:
df = get_schedule_table(start_time='2025-08-11 00:00:00', end_time='2025-08-18 23:59:59')
df

,Location ID,Task Name,Task ID,Robot SN,Map Name,Map URL,Actual Area,Plan Area,Start Time,End Time,...,Progress,Status,Mode,Sub Mode,Type,Vacuum Speed,Vacuum Suction,Wash Speed,Wash Suction,Wash Water
0,530470002,Bullpen -carpet,1146330042331578368,811135422060217,1#2#Bullpen,https://fr-tech-cloud-open.s3.eu-central-1.ama...,125.80,125.80,2025-08-12 18:24:05,2025-08-12 18:34:25,...,100,Task Ended,Sweeping,Unknown,Unknown,Standard,Medium,Off,Off,Off
1,530470002,Bullpen -carpet,1146330042331578368,811135422060217,1#2#Bullpen,https://fr-tech-cloud-open.s3.eu-central-1.ama...,125.80,125.80,2025-07-25 21:55:10,2025-08-11 20:59:36,...,100,Task Ended,Sweeping,Unknown,Unknown,Standard,Medium,Off,Off,Off
2,533370001,Bld_205_1st_Task ALL,1153255765814296576,811135422060216,1#0#Bld_205_Dental_Sci_1fl,https://fr-tech-cloud-open.s3.eu-central-1.ama...,783.78,783.78,2025-08-15 22:14:55,2025-08-15 23:47:09,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,Medium
3,533370001,ground_elevator_new,1153905339323138048,811135422060216,1#1#dental_ground_elevator,https://fr-tech-cloud-open.s3.eu-central-1.ama...,102.32,102.32,2025-08-15 18:06:43,2025-08-15 18:25:46,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,Medium
4,533370001,Bld_205_Grd_Task 1-3,1153152591820505088,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,https://fr-tech-cloud-open.s3.eu-central-1.ama...,52.14,260.71,2025-08-15 17:46:17,2025-08-15 17:57:15,...,20,Task Cancelled,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,Medium
5,533370001,ground_elevator_new,1153905339323138048,811135422060216,1#1#dental_ground_elevator,https://fr-tech-cloud-open.s3.eu-central-1.ama...,6.14,102.32,2025-08-15 17:30:24,2025-08-15 17:32:12,...,6,Task Cancelled,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,Medium
6,533370001,Marston Lib 1st vac+sweep,1153144660123205632,811135422060228,1#0#43_Marston_Library_F1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,739.67,948.30,2025-08-15 11:51:05,2025-08-15 15:06:42,...,78,Task Ended,Sweeping,Custom,Custom,Standard,Medium,Off,Off,Off
7,533370001,Bld_205_Grd_Task ALL,1153524218806681600,811135422060216,-1#0#Bld_205_Dental_Sci_Grd,https://fr-tech-cloud-open.s3.eu-central-1.ama...,793.52,793.52,2025-08-15 12:12:03,2025-08-15 14:03:49,...,100,Task Ended,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,Medium
8,533370001,Marston Lib 1st vac+sweep,1153144660123205632,811135422060228,1#0#43_Marston_Library_F1,https://fr-tech-cloud-open.s3.eu-central-1.ama...,948.30,948.30,2025-08-15 09:11:06,2025-08-15 11:24:13,...,100,Task Ended,Sweeping,Custom,Custom,Standard,Medium,Off,Off,Off
9,533370001,Bld205 Floor2 All,1153611826161270784,811135422060216,2#0#Bld205-Floor2,https://fr-tech-cloud-open.s3.eu-central-1.ama...,0.00,401.07,2025-08-14 22:03:08,2025-08-14 22:03:23,...,0,Task Cancelled,Scrubbing,Custom,Custom,Off,Off,Standard,Medium,Medium


In [ ]:
results = get_cleaning_report_list(start_time='2025-08-11 00:00:00', end_time='2025-08-18 23:59:59', shop_id='533370001',
                                           timezone_offset=0)['list']
results

[{'clean_area': 803.3912353515625,
  'clean_time': 4743,
  'create_time': '2025-08-15 23:47:14',
  'end_time': 1755301629,
  'mac': 'AC:D9:29:88:F8:37',
  'mode': 1,
  'report_id': '1153977308077101056',
  'sn': '811135422060216',
  'start_time': 1755296095,
  'status': 4,
  'sub_mode': 0,
  'task_area': 783.78,
  'task_name': 'Bld_205_1st_Task ALL'},
 {'clean_area': 160.90219116210938,
  'clean_time': 955,
  'create_time': '2025-08-15 18:25:50',
  'end_time': 1755282346,
  'mac': 'AC:D9:29:88:F8:37',
  'mode': 1,
  'report_id': '1153914846480961536',
  'sn': '811135422060216',
  'start_time': 1755281203,
  'status': 4,
  'sub_mode': 0,
  'task_area': 102.3175,
  'task_name': 'ground_elevator_new'},
 {'clean_area': 45.36689376831055,
  'clean_time': 281,
  'create_time': '2025-08-15 17:57:19',
  'end_time': 1755280635,
  'mac': 'AC:D9:29:88:F8:37',
  'mode': 1,
  'report_id': '1153909701508599808',
  'sn': '811135422060216',
  'start_time': 1755279977,
  'status': 6,
  'sub_mode': 0,
 

In [ ]:
df = get_schedule_table(start_time='2025-08-11 00:00:00', end_time='2025-08-18 23:59:59')
df

[]


,Location ID,Task Name,Task ID,Robot SN,Map Name,Map URL,Actual Area,Plan Area,Start Time,End Time,...,Progress,Status,Mode,Sub Mode,Type,Vacuum Speed,Vacuum Suction,Wash Speed,Wash Suction,Wash Water
0,530470002,Bullpen -carpet,1146330042331578368,811135422060217,1#2#Bullpen,https://fr-tech-cloud-open.s3.eu-central-1.ama...,125.8,125.8,2025-08-12 18:24:05,2025-08-12 18:34:25,...,100,Task Ended,Sweeping,Unknown,Unknown,Standard,Medium,Off,Off,Off
1,530470002,Bullpen -carpet,1146330042331578368,811135422060217,1#2#Bullpen,https://fr-tech-cloud-open.s3.eu-central-1.ama...,125.8,125.8,2025-07-25 21:55:10,2025-08-11 20:59:36,...,100,Task Ended,Sweeping,Unknown,Unknown,Standard,Medium,Off,Off,Off


In [ ]:
# Get configuration file path - try multiple locations
import os
config_paths = [
    'database_config.yaml',
    '../src/pudu/configs/database_config.yaml',
    'src/pudu/configs/database_config.yaml',
    'pudu/configs/database_config.yaml',
    '/opt/database_config.yaml'
]

config_path = None
for path in config_paths:
    if os.path.exists(path):
        config_path = path
        break

In [ ]:
app = App(config_path)

In [ ]:
start_time_str = '2025-08-11 00:00:00'
end_time_str = '2025-08-18 23:59:59'

success = app.run(start_time=start_time_str, end_time=end_time_str)

2025-08-11 00:00:00 2025-08-18 23:59:59
Empty DataFrame
Columns: [Location ID, Task Name, Task ID, Robot SN, Map Name, Map URL, Actual Area, Plan Area, Start Time, End Time, Duration, Efficiency, Remaining Time, Consumption, Water Consumption, Progress, Status, Mode, Sub Mode, Type, Vacuum Speed, Vacuum Suction, Wash Speed, Wash Suction, Wash Water]
Index: []

[0 rows x 25 columns]


💥 Critical error in dynamic data pipeline: name 'work_location_success' is not defined
Traceback (most recent call last):
  File "/Users/jiaxuchen/Foxx/Monitor/pudu_robot/src/pudu/app/main.py", line 546, in run
    logger.info(f"🗺️ Work location updates success: {work_location_success}")
NameError: name 'work_location_success' is not defined


NameError: name 'work_location_success' is not defined

In [ ]:
df = get_ongoing_tasks_table()

In [ ]:
df.columns

Index(['location_id', 'task_name', 'task_id', 'robot_sn', 'map_name',
       'map_url', 'actual_area', 'plan_area', 'start_time', 'end_time',
       'duration', 'efficiency', 'remaining_time', 'consumption',
       'water_consumption', 'progress', 'status', 'mode', 'sub_mode', 'type',
       'vacuum_speed', 'vacuum_suction', 'wash_speed', 'wash_suction',
       'wash_water', 'is_report'],
      dtype='object')

In [ ]:
df.to_dict(orient='records')

[{'location_id': 450270000,
  'task_name': 'gym',
  'task_id': '1119987424014385152',
  'robot_sn': '8110H4802050005',
  'map_name': '1#4#church',
  'map_url': '',
  'actual_area': 2.49,
  'plan_area': 249.3,
  'start_time': '2025-08-17 18:08:27',
  'end_time': '2025-08-17 18:27:00',
  'duration': 0,
  'efficiency': 0,
  'remaining_time': 1113,
  'consumption': 0,
  'water_consumption': 0,
  'progress': 1,
  'status': 'Task Interrupted',
  'mode': 'Sweeping',
  'sub_mode': 'Unknown',
  'type': 'Custom',
  'vacuum_speed': 'Standard',
  'vacuum_suction': 'Medium',
  'wash_speed': 'Off',
  'wash_suction': 'Off',
  'wash_water': 'Off',
  'is_report': 1}]

In [ ]:
df.to_dict(orient='records')

[{'location_id': 450270000,
  'task_name': 'gym',
  'task_id': '1119987424014385152',
  'robot_sn': '8110H4802050005',
  'map_name': '1#4#church',
  'map_url': '',
  'actual_area': 2.49,
  'plan_area': 249.3,
  'start_time': '2025-08-17 14:36:26',
  'end_time': '2025-08-17 14:54:59',
  'duration': 0,
  'efficiency': 0,
  'remaining_time': 1113,
  'consumption': 0,
  'water_consumption': 0,
  'progress': 1,
  'status': 'Task Interrupted',
  'mode': 'Sweeping',
  'sub_mode': 'Unknown',
  'type': 'Custom',
  'vacuum_speed': 'Standard',
  'vacuum_suction': 'Medium',
  'wash_speed': 'Off',
  'wash_suction': 'Off',
  'wash_water': 'Off',
  'is_report': 1}]

In [ ]:
get_robot_details('8110H4802050005')

{'battery': 100,
 'cleanbot': {'clean': {'config': {'ai_adaptive_switch': False,
    'left_brush': 0,
    'mode': 2,
    'right_brush': 0,
    'right_vacuum_suction': 0,
    'type': 0,
    'vacuum_speed': 2,
    'vacuum_suction': 2,
    'wash_speed': 0,
    'wash_suction': 0,
    'wash_water': 0},
   'map': {'floor': '1', 'lv': 14, 'name': '1#4#church'},
   'mode': 2,
   'msg': '',
   'report_id': '',
   'result': {'area': 0,
    'break_point': {'clean_type': 2,
     'index': 304,
     'start': None,
     'vector': {'x': -4.901132102022346,
      'y': 11.286307551700224,
      'z': -0.12858972851085665}},
    'charge_count': 0,
    'cost_battery': 0,
    'cost_water': 0,
    'percentage': 1,
    'remaining_time': 1113,
    'status': 3,
    'task_area': 249.29750061035156,
    'time': 0},
   'task': {'name': 'gym',
    'task_id': '1119987424014385152',
    'version': 1747273785400}},
  'detail': '',
  'last_mode': 2,
  'last_task': '1119987424014385152',
  'rising': 0,
  'sewage': 0,
  

In [ ]:
pd.to_datetime(1747273785400, unit='ms')

Timestamp('2025-05-15 01:49:45.400000')

In [ ]:
get_robot_details('8110H4802050005')

{'battery': 100,
 'cleanbot': {'clean': {'config': {'ai_adaptive_switch': False,
    'left_brush': 0,
    'mode': 2,
    'right_brush': 0,
    'right_vacuum_suction': 0,
    'type': 0,
    'vacuum_speed': 2,
    'vacuum_suction': 2,
    'wash_speed': 0,
    'wash_suction': 0,
    'wash_water': 0},
   'map': {'floor': '1', 'lv': 14, 'name': '1#4#church'},
   'mode': 2,
   'msg': '',
   'report_id': '',
   'result': {'area': 0,
    'break_point': {'clean_type': 2,
     'index': 304,
     'start': None,
     'vector': {'x': -4.901132102022346,
      'y': 11.286307551700224,
      'z': -0.12858972851085665}},
    'charge_count': 0,
    'cost_battery': 0,
    'cost_water': 0,
    'percentage': 1,
    'remaining_time': 1113,
    'status': 3,
    'task_area': 249.29750061035156,
    'time': 0},
   'task': {'name': 'gym',
    'task_id': '1119987424014385152',
    'version': 1747273785400}},
  'detail': '',
  'last_mode': 2,
  'last_task': '1119987424014385152',
  'rising': 0,
  'sewage': 0,
  

In [ ]:
pd.to_datetime(1755249066, unit='s')

Timestamp('2025-08-15 09:11:06')

In [ ]:
df = get_schedule_table(start_time='2025-08-11 00:00:00', end_time='2025-08-18 23:59:59')
df

,Location ID,Task Name,Task ID,Robot SN,Map Name,Map URL,Actual Area,Plan Area,Start Time,End Time,...,Progress,Status,Mode,Sub Mode,Type,Vacuum Speed,Vacuum Suction,Wash Speed,Wash Suction,Wash Water
0,530470002,Bullpen -carpet,1146330042331578368,811135422060217,1#2#Bullpen,https://fr-tech-cloud-open.s3.eu-central-1.ama...,125.8,125.8,2025-08-12 18:24:05,2025-08-12 18:34:25,...,100,Task Ended,Sweeping,Unknown,Unknown,Standard,Medium,Off,Off,Off
1,530470002,Bullpen -carpet,1146330042331578368,811135422060217,1#2#Bullpen,https://fr-tech-cloud-open.s3.eu-central-1.ama...,125.8,125.8,2025-07-25 21:55:10,2025-08-11 20:59:36,...,100,Task Ended,Sweeping,Unknown,Unknown,Standard,Medium,Off,Off,Off


In [ ]:
get_cleaning_report_list(start_time='2025-08-16 00:00:00', end_time='2025-08-17 23:59:59', shop_id='450270000', sn='8110H4802050005')

{'limit': 100, 'list': [], 'offset': 0, 'total': 0}

In [ ]:
import timezone

In [ ]:
data = get_cleaning_report_list(start_time='2025-08-13 00:00:00', end_time='2025-08-13 23:59:59', shop_id='533370001', sn='811135422060228')['list']
# convert to datetime
for i in data:
    if i['report_id'] == '1153161039358726144':
        start_time = pd.to_datetime(i['start_time'], unit='s').tz_localize('UTC').tz_convert('US/Eastern')
        end_time = pd.to_datetime(i['end_time'], unit='s').tz_localize('UTC').tz_convert('US/Eastern')
        clean_time = round(i['clean_time'] / 3600, 2)
        print("Report ID: {}; Report start time: {}; Report end time: {}; end_time - start_time: {}; Report's clean_time: {}h;".format(i['report_id'],start_time.strftime('%Y-%m-%d %H:%M:%S'), end_time.strftime('%Y-%m-%d %H:%M:%S'), end_time - start_time, clean_time))
        break

Report ID: 1153161039358726144; Report start time: 2025-08-13 12:11:22; Report end time: 2025-08-13 17:03:39; end_time - start_time: 0 days 04:52:17; Report's clean_time: 2.1h;


In [ ]:

start_time = "2025-08-12 00:00:00"
end_time = "2025-08-15 23:59:59"
charging_table = get_battery_health_list(start_time, end_time, shop_id='533370001')

In [ ]:
import time
int(time.time())

1755387517

In [ ]:
get_list_stores()['list']

[{'company_id': '100004935',
  'company_name': 'Foxx Development Inc.',
  'shop_id': '434100007',
  'shop_name': 'USF demo'},
 {'company_id': '100004935',
  'company_name': 'Foxx Development Inc.',
  'shop_id': '434300005',
  'shop_name': 'USF'},
 {'company_id': '100004935',
  'company_name': 'Foxx Development Inc.',
  'shop_id': '450270000',
  'shop_name': 'salt-lake-airport'},
 {'company_id': '100004935',
  'company_name': 'Foxx Development Inc.',
  'shop_id': '513370000',
  'shop_name': 'Demo'},
 {'company_id': '100004935',
  'company_name': 'Foxx Development Inc.',
  'shop_id': '523670000',
  'shop_name': 'demo-all'},
 {'company_id': '100004935',
  'company_name': 'Foxx Development Inc.',
  'shop_id': '530470002',
  'shop_name': 'Mobiltech'},
 {'company_id': '100004935',
  'company_name': 'Foxx Development Inc.',
  'shop_id': '533370001',
  'shop_name': 'University of Florida'}]

In [ ]:
get_list_robots(shop_id='533370001')

{'count': 2,
 'list': [{'mac': 'AC:D9:29:88:F8:BF',
   'shop_id': '533370001',
   'shop_name': 'University of Florida',
   'sn': '811135422060228'},
  {'mac': 'AC:D9:29:88:F8:37',
   'shop_id': '533370001',
   'shop_name': 'University of Florida',
   'sn': '811135422060216'}]}

In [ ]:
get_cleaning_report_list(start_time='2025-08-15 00:00:00', end_time='2025-08-15 23:59:59', shop_id='533370001', sn='811135422060216')

{'limit': 100,
 'list': [{'clean_area': 803.3912353515625,
   'clean_time': 4743,
   'create_time': '2025-08-15 23:47:14',
   'end_time': 1755301629,
   'mac': 'AC:D9:29:88:F8:37',
   'mode': 1,
   'report_id': '1153977308077101056',
   'sn': '811135422060216',
   'start_time': 1755296095,
   'status': 4,
   'sub_mode': 0,
   'task_area': 783.78,
   'task_name': 'Bld_205_1st_Task ALL'},
  {'clean_area': 160.90219116210938,
   'clean_time': 955,
   'create_time': '2025-08-15 18:25:50',
   'end_time': 1755282346,
   'mac': 'AC:D9:29:88:F8:37',
   'mode': 1,
   'report_id': '1153914846480961536',
   'sn': '811135422060216',
   'start_time': 1755281203,
   'status': 4,
   'sub_mode': 0,
   'task_area': 102.3175,
   'task_name': 'ground_elevator_new'},
  {'clean_area': 45.36689376831055,
   'clean_time': 281,
   'create_time': '2025-08-15 17:57:19',
   'end_time': 1755280635,
   'mac': 'AC:D9:29:88:F8:37',
   'mode': 1,
   'report_id': '1153909701508599808',
   'sn': '811135422060216',
   '

In [ ]:
int(time.time())

1756248454

curl -X POST "http://3.142.93.105:8000/api/pudu/webhook" -H "Content-Type: application/json" -H "CallbackCode: 1vQ6MfUxqyoGMRQ9nK8C4pSkg1Qsa3Vpq" -d '{"callback_type": "notifyRobotPose", "data": {"sn": "811135422060216", "timestamp": 1756248555, "x":1.234, "y":2.345, "yaw":32.34}}'

curl -X POST "http://44.193.26.65:8000/api/pudu/webhook" -H "Content-Type: application/json" -H "CallbackCode: 1vQ6MfUxqyoGMRQ9nK8C4pSkg1Qsa3Vpq" -d '{"callback_type": "robotErrorWarning", "data": {"sn": "811064412050012", "timestamp": 1755387517, "error_type": "LostLocalization", "error_level": "Warning", "error_detail": "",  "error_id": "test00003"}}'

curl -X GET "http://18.222.168.242:8000/api/pudu/webhook/health"

curl -X GET "http://3.142.93.105:8000/api/pudu/webhook/health"

import time

In [ ]:
for result in charging_table['list']:
    print(result['sn'], result['work_status'])

811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060216 11
811135422060228 0
811135422060216 11
811135422060216 11
811135422060216 11
8111354220602

In [ ]:
sns = set()
for result in charging_table['list']:
    sns.add(result['sn'])
sns

{'811135422060216', '811135422060228'}

In [ ]:
battery_health_dict = {}
sn = set()
for health_record in charging_table['list']:
    key = health_record['sn']
    sn.add(key)
    battery_health_dict[key] = {
        'cycle': health_record.get('cycle', None),
        'soc': health_record.get('soc', None),
        'soh': health_record.get('soh', None)
    }

In [ ]:
get_list_robots(shop_id='533370001')['list']

[{'mac': 'AC:D9:29:88:F8:BF',
  'shop_id': '533370001',
  'shop_name': 'University of Florida',
  'sn': '811135422060228'},
 {'mac': 'AC:D9:29:88:F8:37',
  'shop_id': '533370001',
  'shop_name': 'University of Florida',
  'sn': '811135422060216'}]

In [ ]:
sn

{'811135422060216', '811135422060228'}

In [ ]:
results = get_charging_record_list(start_time, end_time, shop_id='533370001', timezone_offset=0)['list']

In [ ]:
sns = set()
for result in results:
    sns.add(result['sn'])
sns

{'811064412050012', '811135422060216', '811135422060228'}

In [ ]:
s = get_charging_table(start_time, end_time, location_id='533370001')
s

,Robot Name,Robot SN,Start Time,End Time,Duration,Initial Power,Final Power,Power Gain,Status
0,Building_205_Dental_Science,811135422060216,2025-08-15 22:14:15,2025-08-16 00:01:22,1h 47min,94%,100%,+6%,Done
1,Building_205_Dental_Science,811135422060216,2025-08-15 20:19:49,2025-08-15 20:20:52,0h 01min,94%,94%,+0%,Done
2,Building_205_Dental_Science,811135422060216,2025-08-15 20:18:45,2025-08-15 21:51:07,1h 32min,51%,94%,+43%,Done
3,Building_43_Marston_Library,811135422060228,2025-08-15 00:15:49,2025-08-15 03:11:19,2h 55min,31%,100%,+69%,Done
4,uf-backup,811064412050012,2025-08-14 21:55:31,2025-08-14 22:00:01,0h 04min,11%,14%,+3%,Done
5,Building_205_Dental_Science,811135422060216,2025-08-14 16:12:11,2025-08-14 16:43:19,0h 31min,94%,100%,+6%,Done
6,Building_205_Dental_Science,811135422060216,2025-08-14 15:36:50,2025-08-14 15:36:50,0h 00min,94%,94%,+0%,Done
7,Building_205_Dental_Science,811135422060216,2025-08-14 14:21:02,2025-08-14 14:21:02,0h 00min,100%,100%,+0%,Done
8,Building_205_Dental_Science,811135422060216,2025-08-13 21:43:40,2025-08-13 21:47:40,0h 04min,34%,35%,+1%,Done
9,uf-backup,811064412050012,2025-08-13 21:29:11,2025-08-13 21:54:41,0h 25min,52%,66%,+14%,Done
